In [26]:
!pip install xmltodict


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [27]:
from requests import Session

import threading
import random
import time

import utils as ut
import html_utils as hu

try:
    ut.fu.copy_folder_structure(
        ut.song_ids_of_song_parts_dir,
        ut.song_parts_from_ids_dir
    )
except Exception as e:
    print(e)

start_letter = 'a'
end_letter = 'z'

song_parts_list = []

error_song_parts_list = []

In [28]:
lock = threading.Lock()

thread_per_batch = 1

thread_list = []

total_saved = 0

In [29]:
class Downloader():
    host = "https://api.hooktheory.com/v1/songs/public/"
    song_parts_item = None
    client = None

    json_file_path = None
    song_link = None

    def __init__(self):
        self.client = Session()

        time_to_sleep = random.uniform(0, 1)
        time.sleep(time_to_sleep)
        
        with lock:
            if len(song_parts_list) > 0:
                self.song_parts_item = song_parts_list.pop(0)
            else:
                self.song_parts_item = None

        if self.song_parts_item is not None:
            self.song_link = self.song_parts_item["link"]
            self.json_file_path = ut.fu.song_link_to_relative_html_file_path(self.song_link)
            self.json_file_path = self.json_file_path.replace(".html", ".json")

            self.json_file_path = ut.fu.relative_to_absolute_path(
                relative_path = self.json_file_path,
                root_path = ut.song_parts_from_ids_dir
            )
        else:
            pass

    def download(self):
        song_part_ids = list(self.song_parts_item["song_parts_ids"])

        song_entire_data = dict(self.song_parts_item)

        song_entire_data["song_parts"] = []
        del song_entire_data["song_parts_ids"]

        print(f"--------> HANDLING url: {self.host}{self.song_parts_item['link']} -> {song_part_ids}")

        for idx in range(len(song_part_ids)):
            song_part_id = song_part_ids[idx]
            print(f"--------> PART {idx} OF {self.song_parts_item['link']}")

            song_part_url = f"{self.host}{song_part_id}"
            
            headers = {
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
                "Accept-Encoding": "gzip, deflate, br",
                "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
                "Connection": "keep-alive",
                "Cookie": "NB_SRVID=srv502887291",
                "Host": "api.hooktheory.com",
                "Sec-Fetch-Dest": "document",
                "Sec-Fetch-Mode": "navigate",
                "Sec-Fetch-Site": "none",
                "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2.1 Safari/605.1.15"
            }            
            
            response = hu.make_request(self.client, song_part_url, headers=headers)

            if response["status_code"] == 200:
                if response["main_data"] is not None:
                    song_entire_data["song_parts"].append(
                        {
                            "song_part_id": song_part_id,
                            "metadata": response["metadata"],
                            "main_data_type": response["main_data_type"],
                            "main_data": response["main_data"]
                        }
                    )

                    time.sleep(random.uniform(2, 5))
                else:
                    error_message = f"FAIL: download - empty main_data: {song_part_url} -> {response}"
                    print(error_message)
                    print()
                    break
            else:
                error_message = f"FAIL: download - status_code: {response['status_code']}: {ut.http_status_code[response['status_code']]}, url: {song_part_url}"
                print(error_message)
                print()
                break

        n_part_downloaded = len(song_entire_data["song_parts"])
        n_part_total = len(song_part_ids)

        if n_part_downloaded == n_part_total:
            ut.fu.write_data_to_json_file(
                song_entire_data,
                self.json_file_path
            )
            
            global total_saved

            with lock:
                total_saved += 1
                print(f"--------> TOTAL SAVED: {total_saved}")
                print

            print(f"--------> SAVED: {self.json_file_path}")
            print(f"SUCCESS: {self.song_link}")
            print()
        else:
            print(f"ACK: {n_part_downloaded}/{n_part_total} part(s) downloaded - {self.song_link}")

            with lock:
                error_song_parts_list.append(self.song_parts_item)
            
            print(f"--------> APPENDED to error_song_parts_list: {self.song_link}")
            print()

    def download_song_parts(self):
        if self.song_parts_item is not None:
            print(f"GETTING: {self.song_parts_item['link']}")
            time.sleep(0.5)
            self.download()
        else:
            print("WARNING: download_song_parts - No more song to handle")
            print()

In [30]:
for letter in range(ord(start_letter), ord(end_letter) + 1):
    song_parts_list.extend(
        ut.read_song_id_of_song_parts_list(
            chr(letter),
            min_json_result_file_size=1
        )
    )

    while len(song_parts_list) > 0:
        for i in range(thread_per_batch):
            downloader = Downloader()
            thread = threading.Thread(target=downloader.download_song_parts)
            thread_list.append(thread)
            thread.start()

        for thread in thread_list:
            thread.join()

        thread_list.clear()

    print()
    print(f"--------> LETTER {chr(letter)} DONE")
    print()


--------> LETTER a DONE


--------> LETTER b DONE


--------> LETTER c DONE


--------> LETTER d DONE


--------> LETTER e DONE


--------> LETTER f DONE


--------> LETTER g DONE


--------> LETTER h DONE


--------> LETTER i DONE


--------> LETTER j DONE


--------> LETTER k DONE


--------> LETTER l DONE


--------> LETTER m DONE


--------> LETTER n DONE


--------> LETTER o DONE


--------> LETTER p DONE


--------> LETTER q DONE


--------> LETTER r DONE


--------> LETTER s DONE


--------> LETTER t DONE


--------> LETTER u DONE


--------> LETTER v DONE


--------> LETTER w DONE


--------> LETTER x DONE


--------> LETTER y DONE


--------> LETTER z DONE



In [31]:
print(error_song_parts_list)
print(len(error_song_parts_list))

[]
0


In [32]:
# song_parts_list = error_song_parts_list

# while len(song_parts_list) > 0:
#     for i in range(thread_per_batch):
#         downloader = Downloader()
#         thread = threading.Thread(target=downloader.download_song_parts)
#         thread_list.append(thread)
#         thread.start()

#     for thread in thread_list:
#         thread.join()

#     thread_list.clear()

In [33]:
song_parts_list_for_validation = []

start_letter_to_validate = 'a'
end_letter_to_validate = 'z'

for letter in range(ord(start_letter_to_validate), ord(end_letter_to_validate) + 1):
    print(f"--------> APPENDING LETTER {chr(letter)}")
    song_parts_list_for_validation.extend(
        ut.read_song_id_of_song_parts_list(
            chr(letter),
            min_json_result_file_size = 9999999999999,
            should_print_log = False
        )
    )

--------> APPENDING LETTER a
--------> APPENDING LETTER b
--------> APPENDING LETTER c
--------> APPENDING LETTER d
--------> APPENDING LETTER e
--------> APPENDING LETTER f
--------> APPENDING LETTER g
--------> APPENDING LETTER h
--------> APPENDING LETTER i
--------> APPENDING LETTER j
--------> APPENDING LETTER k
--------> APPENDING LETTER l
--------> APPENDING LETTER m
--------> APPENDING LETTER n
--------> APPENDING LETTER o
--------> APPENDING LETTER p
--------> APPENDING LETTER q
--------> APPENDING LETTER r
--------> APPENDING LETTER s
--------> APPENDING LETTER t
--------> APPENDING LETTER u
--------> APPENDING LETTER v
--------> APPENDING LETTER w
--------> APPENDING LETTER x
--------> APPENDING LETTER y
--------> APPENDING LETTER z


In [34]:
print(len(song_parts_list_for_validation))
print(song_parts_list_for_validation[1235])

22820


In [41]:
n_data_missing_item = 0

for song_parts_item in song_parts_list_for_validation:
    json_file_path = ut.fu.song_link_to_relative_html_file_path(song_parts_item["link"])
    json_file_path = json_file_path.replace(".html", ".json")
    
    json_file_path = ut.fu.relative_to_absolute_path(
        relative_path = json_file_path,
        root_path = ut.song_parts_from_ids_dir
    )

    song_entire_data = ut.fu.read_data_from_json_file(json_file_path)

    if "key_conclusion" not in song_entire_data:
        n_data_missing_item += 1
        print(f"--------> MISSING key_conclusion: {n_data_missing_item}")
        print(f"                            link: {song_entire_data['link']}")
        print(f"                  json file path: {json_file_path}")
        print()
    else:
        if song_entire_data["key_conclusion"] == '':
            n_data_missing_item += 1
            print(f"--------> EMPTY key_conclusion: {n_data_missing_item}")
            print(f"                          link: {song_entire_data['link']}")
            print(f"                json file path: {json_file_path}")
            print()
        else:
            pass

--------> EMPTY key_conclusion: 1
                          link: https://www.hooktheory.com/theorytab/view/amine/woodlawn
                json file path: /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling/DataPreprocessing/song_parts_from_ids/a/amine/woodlawn.json

--------> EMPTY key_conclusion: 2
                          link: https://www.hooktheory.com/theorytab/view/ariana-grande/let-me-love-you-ft-lil-wayne
                json file path: /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling/DataPreprocessing/song_parts_from_ids/a/ariana-grande/let-me-love-you-ft-lil-wayne.json

--------> EMPTY key_conclusion: 3
                          link: https://www.hooktheory.com/theorytab/view/ariana-grande/the-way-ft-mac-miller
                json file path: /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling/DataPreprocessing/song_parts_from_ids/a/ariana-grande/the-way-ft-mac-miller.json

--------> E